In [9]:
# Install necessary libraries
# !pip install gensim


In [19]:
import gensim.downloader as api

model_name = 'glove-wiki-gigaword-50'
model = api.load(model_name)

print(f"Model '{model_name}' downloaded successfully.")

[==================================================] 100.0% 66.0/66.0MB downloaded
Model 'glove-wiki-gigaword-50' downloaded successfully.


In [10]:
model

In [11]:
from gensim.models import KeyedVectors
import pandas as pd

In [20]:
# Convert the model to a pandas DataFrame
word_vectors = model.vectors
words = model.index_to_key
df = pd.DataFrame(word_vectors, index=words)

print(f"\nDataFrame created successfully. {df.shape=}")
df.head()


DataFrame created successfully. df.shape=(400000, 50)


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
the,0.418000,0.249680,-0.41242,0.12170,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,-0.656600,...,-0.298710,-0.157490,-0.347580,-0.045637,-0.44251,0.187850,0.002785,-0.184110,-0.115140,-0.78581
",",0.013441,0.236820,-0.16899,0.40951,0.63812,0.477090,-0.42852,-0.55641,-0.36400,-0.239380,...,-0.080262,0.630030,0.321110,-0.467650,0.22786,0.360340,-0.378180,-0.566570,0.044691,0.30392
.,0.151640,0.301770,-0.16763,0.17684,0.31719,0.339730,-0.43478,-0.31086,-0.44999,-0.294860,...,-0.000064,0.068987,0.087939,-0.102850,-0.13931,0.223140,-0.080803,-0.356520,0.016413,0.10216
of,0.708530,0.570880,-0.47160,0.18048,0.54449,0.726030,0.18157,-0.52393,0.10381,-0.175660,...,-0.347270,0.284830,0.075693,-0.062178,-0.38988,0.229020,-0.216170,-0.225620,-0.093918,-0.80375
to,0.680470,-0.039263,0.30186,-0.17792,0.42962,0.032246,-0.41376,0.13228,-0.29847,-0.085253,...,-0.094375,0.018324,0.210480,-0.030880,-0.19722,0.082279,-0.094340,-0.073297,-0.064699,-0.26044


In [21]:
df.loc["india"].values

array([-0.20356 , -0.8707  , -0.19172 ,  0.73862 ,  0.18494 ,  0.14926 ,
        0.48079 , -0.21633 ,  0.72753 , -0.36912 ,  0.13397 , -0.1143  ,
       -0.18075 , -0.64683 , -0.18484 ,  0.83575 ,  0.48179 ,  0.76026 ,
       -0.50381 ,  0.80743 ,  1.2195  ,  0.3459  ,  0.22185 ,  0.31335 ,
        1.2066  , -1.8441  ,  0.14064 , -0.99715 , -1.1402  ,  0.32342 ,
        3.2128  ,  0.42708 ,  0.19504 ,  0.80113 ,  0.38555 , -0.12568 ,
       -0.26533 ,  0.055264, -1.1557  ,  0.16836 , -0.82228 ,  0.20394 ,
        0.089235, -0.60125 , -0.032878,  1.3735  , -0.51661 ,  0.29611 ,
        0.23951 , -1.3801  ], dtype=float32)

In [27]:
# prompt: from the df filter the words that have non alpha charater like a sybom and digit

import re

def is_alpha_only(word):
  """Checks if a word contains only alphabetic characters."""
  return bool(re.fullmatch(r'[a-zA-Z]+', word))

# Filter the DataFrame index (words) to keep only alpha-only words
alpha_only_words = [word for word in df.index if is_alpha_only(word)]

# Create a new DataFrame with only the filtered words
df_alpha_only = df.loc[alpha_only_words]

print(f"\nDataFrame filtered to include only alpha-only words. {df_alpha_only.shape=}")


DataFrame filtered to include only alpha-only words. df_alpha_only.shape=(317756, 50)


In [28]:
# take random sample of 8000 orws from the df, with a random seed 42
df_sample = df_alpha_only.sample(n=4000, random_state=42)
df_sample.shape

(4000, 50)

In [29]:
# prompt: reduce the dimenssion of the vectors to 2d using UMAP

# !pip install umap-learn

import umap
reducer = umap.UMAP(n_components=2, random_state=42)

# Fit and transform the sampled data
embedding = reducer.fit_transform(df_sample)

# Create a new DataFrame for the 2D embedding
df_embedding = pd.DataFrame(embedding, index=df_sample.index, columns=['UMAP1', 'UMAP2'])

print("UMAP dimensionality reduction complete.")
print(f"Embedding DataFrame shape: {df_embedding.shape}")


UMAP dimensionality reduction complete.
Embedding DataFrame shape: (4000, 2)


## Plot the vectors (first two components) using plotly

In [15]:
import plotly.express as px

In [30]:
px.scatter(df_embedding, x="UMAP1", y="UMAP2", hover_name=df_embedding.index)